In [1]:
import pandas as pd
import json 
import numpy as np
import scipy
import random
from data_model import *
from pprint import pprint
import re

/Users/User/.virtualenvs/window_study/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
from functools import partial

def rand_gen(mu,sigma):
    return sigma * np.random.randn() + mu



In [3]:
task_type_time = {t_type.task_type_id:partial(rand_gen, mu = t_type.expected_time,sigma=np.random.randn()+1) for t_type in TaskType.select(TaskType.task_type_id,TaskType.expected_time).order_by(TaskType.task_type_id)}

In [4]:
task_type_time[1]()

5.47738292072486

In [5]:
task_type_pay = {t_type.task_type_id:partial(rand_gen, mu = t_type.expected_pay,sigma=np.random.randn()*0.001) for t_type in TaskType.select(TaskType.task_type_id,TaskType.expected_pay).order_by(TaskType.task_type_id)}

In [12]:
pattern = "(http|ftp|https)?(://)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
link_finder = re.compile(pattern)


In [16]:
def extract_amounts(task,task_type):
    pattern = "(http|ftp|https)?(://)?([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?"
    link_finder = re.compile(pattern)
    num_images = 0
    num_words = 0
    num_links = 0
    num_options = 0
    if 'text' in task:
        for text_objects in task['text']:
            num_links += len(link_finder.findall(str(text_objects['content'])))
            num_words += len(str(text_objects['content']))
    if 'image' in task:
        num_images = len(task['image'])
    
    if "choice" in task:
        for opt in task['choice']:
            num_options += len(opt['options'])
    
    new_t = {'num_of_words':num_words,
            'num_of_images':num_images,
            'num_of_links':num_links,
            'num_options' : num_options,
            'expected_time':task_type_time[task_type](),
            'expected_pay' :task_type_pay[task_type]()
            }  
    return new_t
        

In [17]:
tasks = Task.select()
for t in tasks.iterator():
    if t.id % 1000 == 0:
        print("*",end="")
    if t.id % 10000 == 0:
        print()

    feat = extract_amounts(t.content,t.task_type.task_type_id)
    TaskFeatures.create(task_id=t.id,feature = feat)

*********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
**********
*********

In [15]:
for text_objects in t.content['text']:
    print(text_objects)

{'content': 419, 'label': 'Name of App'}
{'content': "Jump, fly and dodge through Antarctica. Air Penguin is incredibly simple and highly addictive. All you need to know is how to tilt.\nJourney through the icy South Pole to help Air Penguin save his family from melting ice caps.\n\nWho said penguins can't fly?! Make them flap their tiny wings!\nTilt your way out of danger as you bounce, glide and slide through this Antarctic adventure today!\n\n\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\n\nCRITICS SAY\n\nAir Penguin's hotter than Tiny Wings and topping Angry Birds.\n- Kotaku\n\nAir Penguin is fun, no doubt about it. Addictive, too, pushing all the right  just one more time  buttons.\n- CNET\n\nAir Penguin has adorable graphics, dead simple gameplay, and an engaging scoring system that ensures you\x89Ûªll keep coming back for more.\n- Touch Arcade\n\n\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89ÓÛ\x89Ó

In [119]:
extract_amounts(sample_tasks[0].content,sample_tasks[0].task_type.task_type_id)

{'num_of_words': 69,
 'num_of_images': 0,
 'num_of_links': 0,
 'num_options': 10,
 'expected_time': 28.33029081518132,
 'expected_pay': 0.09080032548536406}

In [123]:
for t in sample_tasks:
    pprint(extract_amounts(t.content,t.task_type.task_type_id))
    print('*'*100)

{'expected_pay': 0.08999775047992173,
 'expected_time': 28.56151381293192,
 'num_of_images': 0,
 'num_of_links': 0,
 'num_of_words': 69,
 'num_options': 10}
****************************************************************************************************
{'expected_pay': 0.07980876633776919,
 'expected_time': 26.687037629459013,
 'num_of_images': 1,
 'num_of_links': 0,
 'num_of_words': 0,
 'num_options': 17}
****************************************************************************************************
{'expected_pay': 0.10009849028070536,
 'expected_time': 34.44564491469758,
 'num_of_images': 1,
 'num_of_links': 0,
 'num_of_words': 0,
 'num_options': 0}
****************************************************************************************************
{'expected_pay': 0.0995689451669651,
 'expected_time': 35.7842385391175,
 'num_of_images': 1,
 'num_of_links': 0,
 'num_of_words': 0,
 'num_options': 0}
***************************************************************************